# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096919


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:05<01:20,  2.88s/it]

Rendering models:  13%|█▎        | 4/30 [00:06<00:54,  2.08s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:50,  2.02s/it]

Rendering models:  20%|██        | 6/30 [00:08<00:37,  1.58s/it]

...

Rendering models:  87%|████████▋ | 26/30 [00:16<00:01,  2.62it/s]

Rendering models:  90%|█████████ | 27/30 [00:16<00:01,  2.81it/s]

Rendering models:  93%|█████████▎| 28/30 [00:16<00:00,  2.57it/s]

Rendering models: 100%|██████████| 30/30 [00:17<00:00,  2.86it/s]

not-logged-in-673e227f920b24558cc1    0.000326
sorenjensen01                         0.000223
Jonas_Cross                           0.000625
2530289                               0.055756
Jaronec                               0.000237
ElisabethB                            0.000266
not-logged-in-c6917d9b19f41e6c2995    0.001901
gracie_solveig                        0.000324
babyduckyt                            0.088627
not-logged-in-f7e2260ed4a9517a3e4a    0.000816
not-logged-in-aeab25becc0fe7de0137    0.036218
Galaxy-one                            0.091369
sn346808                              0.034847
sn345012                              0.004449
not-logged-in-14d09f70263c1fe15cc3    0.000392
engrg                                 0.000301
puic                                  0.001506
Donihee                               0.004516
not-logged-in-4ac7172b782ab7b789bc    0.121627
umtuck1                               0.000356
HenriqueDornelles                     0.040397
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())